<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/Limpieza_ACT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Limpieza Actividades

In [21]:
!pip install unidecode

import os
import pandas as pd
import numpy as np
from unidecode import unidecode
import matplotlib.pyplot as plt

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
# Asegúrarse de cambiar la ruta por el nombre real de la carpeta en Google Drive
folder_path = '/content/drive/My Drive/PFG_FASPAS/ACT'
docs_xlsx = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
print(docs_xlsx)

['GU_ACT.xlsx']


In [24]:
dic_dataframes = {}

for doc in docs_xlsx:
    entire_path = os.path.join(folder_path, doc)
    df = pd.read_excel(entire_path)
    # Uso el nombre del archivo como clave
    dic_dataframes[doc] = df

In [25]:
# Imprimir todas las claves
for clave in dic_dataframes.keys():
    print(clave)

GU_ACT.xlsx


In [26]:
dic_dataframes['GU_ACT.xlsx']

,FECHA,ACTIVIDAD,LOCALIDAD,CP,PARTICIPANTES
0,2019-02-13,Formación centro educativo,Marchamalo,19180.0,27
1,2019-03-02,Sensibilización Día Implante Coclear,Guadalajara,19003.0,37
2,2019-03-06,Formación centro educativo,Guadalajara,19002.0,10
3,2019-03-13,Reunión proyecto colaborativo Ponte en mi lugar,Cabanillas del Campo,19171.0,10
4,2019-03-28,Formación centro de mayores,Guadalajara,19004.0,20
...,...,...,...,...,...
143,2023-10-19,Presentación libro: préstamo bucle,Guadalajara,19001.0,20
144,2023-10-23,Sensibilización CEIP,Guadalajara,19001.0,436
145,2023-11-04,Sensibilización CEIP,Torija,19190.0,55
146,2023-11-23,Día de la discapacidad y voluntariado,Guadalajara,19001.0,60


In [27]:
# Definir la función que realiza las transformaciones
def transform_df(df):
    col_excluded = ['FECHA']

    # Convertir todas las columnas de tipo object a mayúsculas, excepto las especificadas
    for column in df.columns:
        if df[column].dtype == 'object' and column not in col_excluded:
            df[column] = df[column].apply(lambda x: unidecode(x.upper()) if isinstance(x, str) else x)

    # Ahora, aplicamos unidecode a los nombres de las columnas
    df.columns = [unidecode(col.upper()) for col in df.columns]

    return df

def CP_year(df):
    # Convertir a numérico, dejando como NaN los valores que no sean convertibles
    df['CP'] = pd.to_numeric(df['CP'], errors='coerce').astype('Int64')

    df = df.dropna(subset=['CP', 'LOCALIDAD'], how='all')
    CP_loc_filter = df['CP'].isnull() & df['LOCALIDAD'].isnull()
    df = df[~CP_loc_filter]

    return df

def other_columns(df):
  if 'LOCALIDAD' in df.columns:
    # Limpiar los datos eliminando contenido entre paréntesis
    df['LOCALIDAD'] = df['LOCALIDAD'].str.replace(r'\s*\([^)]*\)', '', regex=True)

  if 'FECHA' in df.columns:
    df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
    df['YEAR'] = df['FECHA'].dt.year
    df.drop('FECHA', axis=1, inplace=True)

  return df

In [28]:
# Suponiendo que 'dict_of_dfs' es tu diccionario de DataFrames
for df_key, df in dic_dataframes.items():
    dic_dataframes[df_key] = transform_df(df)
    dic_dataframes[df_key] = CP_year(df)
    dic_dataframes[df_key] = other_columns(df)

In [29]:
GU_ACT = dic_dataframes.get("GU_ACT.xlsx")
GU_ACT

,ACTIVIDAD,LOCALIDAD,CP,PARTICIPANTES,YEAR
0,FORMACION CENTRO EDUCATIVO,MARCHAMALO,19180,27,2019
1,SENSIBILIZACION DIA IMPLANTE COCLEAR,GUADALAJARA,19003,37,2019
2,FORMACION CENTRO EDUCATIVO,GUADALAJARA,19002,10,2019
3,REUNION PROYECTO COLABORATIVO PONTE EN MI LUGAR,CABANILLAS DEL CAMPO,19171,10,2019
4,FORMACION CENTRO DE MAYORES,GUADALAJARA,19004,20,2019
...,...,...,...,...,...
143,PRESENTACION LIBRO: PRESTAMO BUCLE,GUADALAJARA,19001,20,2023
144,SENSIBILIZACION CEIP,GUADALAJARA,19001,436,2023
145,SENSIBILIZACION CEIP,TORIJA,19190,55,2023
146,DIA DE LA DISCAPACIDAD Y VOLUNTARIADO,GUADALAJARA,19001,60,2023


In [30]:
CLM_ACT = GU_ACT

In [31]:
# List of DataFrames
dfs = [GU_ACT, CLM_ACT]

# Corresponding folder paths on Google Drive
folder_paths = [
    '/content/drive/My Drive/PFG_FASPAS/GU',
    '/content/drive/My Drive/PFG_FASPAS/CLM'
]

# Corresponding file names
file_names = ['GU_ACT_limpio.xlsx', 'CLM_ACT_limpio.xlsx']

In [32]:
# Iterate over the DataFrames, folder paths, and file names
for df, folder_path, file_name in zip(dfs, folder_paths, file_names):
    # Define the complete file path
    file_path = f"{folder_path}/{file_name}"

    # Save the DataFrame as an Excel file
    df.to_excel(file_path, index=False)

print("All DataFrames have been exported successfully.")

All DataFrames have been exported successfully.
